# Dependencies

In [1]:
from sys import path
path.append('src/')

import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from src.mrk_file import MRKFile
from src.iso_standard import PhotographicRequirements
from keras.applications import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model, Input

%matplotlib inline

Using TensorFlow backend.


In [2]:
FILE_TRAIN_DUMP = 'data/train_dump.pkl'
FILE_TRAIN_BOTTLENECKS = 'data/train_bottlenecks.pkl'

FILE_VAL_DUMP = 'data/val_dump.pkl'
FILE_VAL_BOTTLENECKS = 'data/val_bottlenecks.pkl'

INPUT_SHAPE = (224, 224, 3)

# Load Data 

In [4]:
train_image_files, x_train, train_mrks = pkl.load(open(FILE_TRAIN_DUMP, 'rb'))

print(len(train_image_files), type(train_image_files))
print(x_train.shape, x_train.dtype)
print(len(train_mrks), type(train_mrks[0]))

5211 <class 'list'>
(5211, 224, 224, 3) float32
5211 <class 'src.mrk_file.MRKFile'>


In [5]:
val_image_files, x_val, val_mrks = pkl.load(open(FILE_VAL_DUMP, 'rb'))

print(len(val_image_files), type(val_image_files))
print(x_val.shape, x_val.dtype)
print(len(val_mrks), type(val_mrks[0]))

565 <class 'list'>
(565, 224, 224, 3) float32
565 <class 'src.mrk_file.MRKFile'>


# Save Bottlenecks 

In [5]:
base_model = MobileNet(input_shape=INPUT_SHAPE, weights='imagenet', include_top=False)

In [6]:
train_features = base_model.predict(x_train, batch_size=32, verbose=1)
val_features = base_model.predict(x_val, batch_size=32, verbose=1)

print(train_features.shape, train_features.dtype)
print(val_features.shape, val_features.dtype)

565/565 [==============================] - 1s 1ms/step
(5211, 7, 7, 1024) float32
(565, 7, 7, 1024) float32


In [7]:
y_train = np.array([mrk.photo_reqs.values() for mrk in train_mrks], dtype=np.int8)
y_val = np.array([mrk.photo_reqs.values() for mrk in val_mrks], dtype=np.int8)

print(y_train.shape, y_train.dtype)
print(y_val.shape, y_val.dtype)

(5211, 23) int8
(565, 23) int8


In [8]:
pkl.dump((train_features, y_train), open(FILE_TRAIN_BOTTLENECKS, 'wb'), protocol=-1)
pkl.dump((val_features, y_val), open(FILE_VAL_BOTTLENECKS, 'wb'), protocol=-1)

# Transfer Learning 

In [3]:
train_features, y_train = pkl.load(open(FILE_TRAIN_BOTTLENECKS, 'rb'))
val_features, y_val = pkl.load(open(FILE_VAL_BOTTLENECKS, 'rb'))

print(train_features.shape, train_features.dtype)
print(y_train.shape, y_train.dtype)
print(val_features.shape, val_features.dtype)
print(y_val.shape, y_val.dtype)

(5211, 7, 7, 1024) float32
(5211, 23) int8
(565, 7, 7, 1024) float32
(565, 23) int8


In [4]:
inputs = Input(shape=train_features.shape[1:], name='inputs')
avg_pool = GlobalAveragePooling2D()(inputs)

r2 = Dense(units=3, activation='softmax', name='output_blurred')(avg_pool)
r3 = Dense(units=3, activation='softmax', name='output_looking_away')(avg_pool)
r4 = Dense(units=3, activation='softmax', name='output_ink_marked_creased')(avg_pool)
r5 = Dense(units=3, activation='softmax', name='output_unnatural_skin_tone')(avg_pool)
r6 = Dense(units=3, activation='softmax', name='output_too_dark_light')(avg_pool)
r7 = Dense(units=3, activation='softmax', name='output_washed_out')(avg_pool)
r8 = Dense(units=3, activation='softmax', name='output_pixelation')(avg_pool)
r9 = Dense(units=3, activation='softmax', name='output_hair_across_eyes')(avg_pool)
r10 = Dense(units=3, activation='softmax', name='output_eyes_closed')(avg_pool)
r11 = Dense(units=3, activation='softmax', name='output_varied_background')(avg_pool)
r12 = Dense(units=3, activation='softmax', name='output_roll_pitch_yaw')(avg_pool)
r13 = Dense(units=3, activation='softmax', name='output_flash_reflection_on_skin')(avg_pool)
r14 = Dense(units=3, activation='softmax', name='output_red_eyes')(avg_pool)
r15 = Dense(units=3, activation='softmax', name='output_shadows_behind_head')(avg_pool)
r16 = Dense(units=3, activation='softmax', name='output_shadows_across_face')(avg_pool)
r17 = Dense(units=3, activation='softmax', name='output_dark_tinted_lenses')(avg_pool)
r18 = Dense(units=3, activation='softmax', name='output_flash_reflection_on_lenses')(avg_pool)
r19 = Dense(units=3, activation='softmax', name='output_frames_too_heavy')(avg_pool)
r20 = Dense(units=3, activation='softmax', name='output_frame_covering_eyes')(avg_pool)
r21 = Dense(units=3, activation='softmax', name='output_hat_cap')(avg_pool)
r22 = Dense(units=3, activation='softmax', name='output_veil_over_face')(avg_pool)
r23 = Dense(units=3, activation='softmax', name='output_mouth_open')(avg_pool)
r24 = Dense(units=3, activation='softmax', name='output_presence_of_other_faces_or_toys')(avg_pool)

model = Model(inputs=inputs, 
              outputs=[r2, r3, r4, r5, r6, r7, r8, r9, r10, r11, r12, r13, r14, r15, r16, r17, r18, r19, r20, r21, r22, r23, r24], 
              name='icaonet')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             (None, 7, 7, 1024)   0                                            
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 1024)         0           inputs[0][0]                     
__________________________________________________________________________________________________
output_blurred (Dense)          (None, 3)            3075        global_average_pooling2d_1[0][0] 
__________________________________________________________________________________________________
output_looking_away (Dense)     (None, 3)            3075        global_average_pooling2d_1[0][0] 
__________________________________________________________________________________________________
output_ink

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.fit(train_features, np.hsplit(y_train, range(1, y_train.shape[1])), batch_size=32, epochs=1)

Epoch 1/1
5211/5211 [==============================] - 6s 1ms/step - loss: nan - output_blurred_loss: nan - output_looking_away_loss: nan - output_ink_marked_creased_loss: nan - output_unnatural_skin_tone_loss: nan - output_too_dark_light_loss: nan - output_washed_out_loss: nan - output_pixelation_loss: nan - output_hair_across_eyes_loss: nan - output_eyes_closed_loss: nan - output_varied_background_loss: nan - output_roll_pitch_yaw_loss: nan - output_flash_reflection_on_skin_loss: nan - output_red_eyes_loss: nan - output_shadows_behind_head_loss: nan - output_shadows_across_face_loss: nan - output_dark_tinted_lenses_loss: 0.2926 - output_flash_reflection_on_lenses_loss: nan - output_frames_too_heavy_loss: nan - output_frame_covering_eyes_loss: nan - output_hat_cap_loss: nan - output_veil_over_face_loss: 0.1697 - output_mouth_open_loss: nan - output_presence_of_other_faces_or_toys_loss: nan - output_blurred_acc: 0.1418 - output_looking_away_acc: 0.1128 - output_ink_marked_creased_acc: 